In [2]:
deployment = "gpt4"
model = "gpt-4"

# 大模型辅助设计

In [2]:
import openai
def work_on(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
        model=model,
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 500
      )
    return response.choices[0].message.content

In [3]:
prompt = """
Design a microservice according to the following requirements:
The service will process the user's input and store the result. 
After processing a token will be returned to the user, it would be used to get the processing result.
Consider on the following design principles:
1. concerns separation, try you best to separate the functions into the small modules.
2. single responsibility 
3. interface oriented
make sure the extensibility and reusability of the design, represent your design (with main abstractions) with PlantUML script
"""

In [4]:
print(work_on(prompt))

Below is a basic design of the microservice:

The microservice will be split into the following modules:

1. **InputProcessor:** This module will be responsible for processing the user's input. Interface-oriented design will be followed to ensure extensibility and reusability.

2. **TokenGenerator:** This module will generate a unique token for each processed input. This design allows for separation of concerns and follows the single responsibility principle.

3. **ResultStore:** This module will store the processed result against the generated token. This design also follows the single responsibility principle.

4. **TokenValidator:** This module will validate the provided token and fetch the corresponding result.

Here is the PlantUML script representing the above design:

```plantuml
@startuml
title Microservice Design

class InputProcessor {
  +process(input: Input): Result
}

class TokenGenerator {
  +generateToken(result: Result): Token
}

class ResultStore {
  +store(token: Toke

# 生成Plugin代码及Skeleton单元测试

In [3]:
import os
import json
import re

def find_declarations(directory, json_string):
    # Parse the JSON string
    json_data = json.loads(json_string)
    package_name = json_data["package_name"]
    element_name = json_data["element"]

    # Regular expression to match structure/interface declarations
    regex = re.compile(r"(type\s+" + element_name + r"\s+(struct|interface)\s+{\s+[^}]*\s+})")

    # Read the source files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".go"):
                with open(os.path.join(root, file), "r") as f:
                    content = f.read()
                    # Check if the file is in the correct package
                    if "package " + package_name in content:
                        # Search for the structure/interface declaration
                        match = regex.search(content)
                        if match:
                            return match.group(1)

    return "Not Found"

In [4]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
import time
import os
import json
import re

src_dir = "./llm_friendly_code"

def find_declarations(directory, json_string):
    # Parse the JSON string
    json_data = json.loads(json_string)
    package_name = json_data["package_name"]
    element_name = json_data["element"]

    # Regular expression to match structure/interface declarations
    regex = re.compile(r"(type\s+" + element_name + r"\s+(struct|interface)\s+{\s+[^}]*\s+})")

    # Read the source files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".go"):
                with open(os.path.join(root, file), "r") as f:
                    content = f.read()
                    # Check if the file is in the correct package
                    if "package " + package_name in content:
                        # Search for the structure/interface declaration
                        match = regex.search(content)
                        if match:
                            return match.group(1)

    return "Not Found"

def get_dep(json_string):
    return find_declarations(src_dir, json_string)

llm = AzureChatOpenAI(deployment_name=deployment, temperature=0)

# 将“依赖获取”工具添加到 ReAct 过程中    
tools = [
    Tool(
        name="find_declarations", func=get_dep,
        description="""Only invoke it when you need to find the declaration of the interface/struct/method that is not in the given code. 
        The input is a JSON string includes package name and the struct/interface name, 
          and its the format {{"package_name":<package name>,"element":<struct/interface>}}
        """)
]

In [6]:
def write_code(input):
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", handle_parsing_errors=True, max_iterations=10, verbose=True)
    print(agent.run(input))

In [7]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.TokenCreator interface.
 The implementation is to generate the token with uuid and its package is demo
"""
)



> Entering new AgentExecutor chain...
First, I need to know what methods are in the demo.TokenCreator interface. 
Action: find_declarations
Action Input: {"package_name":"demo","element":"TokenCreator"}
Observation: type TokenCreator interface {
	CreateToken(data string) string
}
Thought:I now know the final answer
Final Answer: 
Here is the implementation of the demo.TokenCreator interface in Go:

```go
package demo

import (
	"github.com/google/uuid"
)

type MyTokenCreator struct{}

func (m *MyTokenCreator) CreateToken(data string) string {
	return uuid.New().String() + data
}
```

In this code, we define a struct `MyTokenCreator` that will implement the `TokenCreator` interface. We then define a method `CreateToken` on `MyTokenCreator` that takes a string as input and returns a string. This method generates a new UUID and appends the input data to it to create the token.

> Finished chain.
Here is the implementation of the demo.TokenCreator interface in Go:

```go
package demo

im

In [8]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.Storage interface.
 The implementation is based on local memory.
"""
)



> Entering new AgentExecutor chain...
First, I need to know what methods are in the demo.Storage interface.
Action: find_declarations
Action Input: {"package_name":"demo","element":"Storage"}
Observation: type Storage interface {
	// RetiveData is to retrive the data by the associated token.
	RetieveData(token string) (string, error)

	// StoreData is to persist the data,
	// input paramters:
	//   token is used to retrive the associated data
	//
	StoreData(token string, data string) error
}
Thought:I now know the methods in the demo.Storage interface. I need to implement these methods in a struct that represents local memory storage.
Final Answer: 
Here is the Go code to implement the demo.Storage interface based on local memory:

```go
package main

import (
	"errors"
	"sync"
)

type LocalMemoryStorage struct {
	data map[string]string
	mu   sync.RWMutex
}

func NewLocalMemoryStorage() *LocalMemoryStorage {
	return &LocalMemoryStorage{
		data: make(map[string]string),
	}
}

func (l 

In [9]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.Processor interface.
 The implementation is to reverse the input content
"""
)



> Entering new AgentExecutor chain...
First, I need to know what the demo.Processor interface looks like in order to implement it. 
Action: find_declarations
Action Input: {"package_name":"demo","element":"Processor"} 
Observation: type Processor interface {
	// Process is to the raw data
	Process(raw string) (string, error)
}
Thought:Now I know the structure of the demo.Processor interface. It has a single method, Process, which takes a raw string as input and returns a string and an error. I can now write the implementation of this interface. The implementation should reverse the input string.
Final Answer: Here is the Go code to implement the demo.Processor interface:

```go
package main

import (
	"errors"
	"strings"
)

type ReverseProcessor struct{}

func (rp *ReverseProcessor) Process(raw string) (string, error) {
	if raw == "" {
		return "", errors.New("input string is empty")
	}

	runes := []rune(raw)
	for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
		runes[i], runes[j]

In [10]:
def write_unit_tests(filepath):
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", handle_parsing_errors=True, max_iterations=10, verbose=True)
    return (agent.run(

read_file(filepath) +        
"""\n
    ----\n
你是一个资深软件工程师，请为上面的代码编写单元测试，按下面步骤进行：

1. 测试程序与被测试程序在同一 package 中

2. 分析需要测试的源码

3. 先在遗留代码中查找写测试所需要的接口的声明，便于生成 mock 对象

根据以上输出单元测试程序
    
    \n ---""" + read_file(filepath)
     ))

In [11]:
print(write_unit_tests(src_dir+"/demo/skeleton.go"))



> Entering new AgentExecutor chain...
The code provided is a service implementation with two methods: Process and Retrieve. These methods use three interfaces: Processor, TokenCreator, and Storage. To write unit tests for this service, I need to create mock implementations of these interfaces. However, the declarations of these interfaces are not provided in the code. I need to find the declarations of these interfaces first.

Action: find_declarations
Action Input: {"package_name":"demo","element":"Processor"}
Observation: type Processor interface {
	// Process is to the raw data
	Process(raw string) (string, error)
}
Thought:Now I have the declaration of the Processor interface. I need to find the declarations of the TokenCreator and Storage interfaces next.

Action: find_declarations
Action Input: {"package_name":"demo","element":"TokenCreator"}
Observation: type TokenCreator interface {
	CreateToken(data string) string
}
Thought:Now I have the declaration of the TokenCreator inte